# Data Pre Processing

This is a notebook for intaking a raw data file, preprocessing it, and exporting it in a format in which it can easily be labeled.

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("data/directoryident.tsv", sep="\t")
df.head()

,dn,lastname,givenname,middle,dob,address
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,NaN
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,NaN,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,NaN,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,NaN,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,NaN,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...


## Preprocessing

### Cleanup

In [4]:
# Add unique ID column
# Uses the row number of original csv file as unique ID
df['Unique ID'] = df.index + 1

In [5]:
# Turn all NaN values into empty strings
df['address'] = df['address'].fillna(" ")
df['middle'] = df['middle'].fillna(" ")

In [6]:
df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,,1
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...,2
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...,3
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...,4
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...,5


In [7]:
# Convert all columns to string
df = df.astype(str)

In [8]:
# take sample of 5000
df = df.sample(n=5000)

In [9]:
# Function to parse a singular dn field
# Returns the uid in the dn
def parse_dn(dn):
    dn = dn.split(',')
    dn = [x.split('=') for x in dn]
    dn = {x[0].strip(): x[1].strip() for x in dn}
    return dn["uid"]

In [10]:
# Function to concatenate every value in the address 
# We want to concatenate all values inside curly braces
def concat_address(address):
    if address != ' ':
        address = address.split('}:{')
        try: 
            address = [x.split('=')[1] for x in address]
        except:
            return "DELETE"
        address = ' '.join(address)
    return address

In [11]:
# Use vectorization to set new values for dn and address columns
df['dn'] = df['dn'].apply(parse_dn)
df['address'] = df['address'].apply(concat_address)
df['dob'] = df['dob'].apply(lambda x : x[:8])

In [12]:
if "DELETE" in df['address'].values:
    df = df[df['address'] != "DELETE"]

df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
126041,dgberton,Bertoni,Dylan,,19930625,837 W Lombard St Baltimore MD 21201 UNITED STA...,126042
271929,patwalk,Walker,Patrick,J,19950125,490 M St SW Apt W603 Washington DC 20024-2628 ...,271930
30365,ricraig,RICHARD,CRAIG,,nan,,30366
578243,jochne,Neuenschwander,John,C,19621228,1508 Ottawa Rd Ann Arbor MI 48105 USA},578244
182283,changfel,Liu,Changfeng,,19940724,"Room 1401, No. 12, 99 Nong Jinggu East RD. Min...",182284


In [13]:
# Change column names
df.columns = ['ID', 'Last Name', 'First Name', 'Middle Name', 'Date of Birth', 'Address', 'Unique ID']
# Reorder columns
df = df[['Unique ID', 'ID', 'First Name', 'Middle Name', 'Last Name', 'Date of Birth', 'Address']]

In [14]:
df.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
126041,126042,dgberton,Dylan,,Bertoni,19930625,837 W Lombard St Baltimore MD 21201 UNITED STA...
271929,271930,patwalk,Patrick,J,Walker,19950125,490 M St SW Apt W603 Washington DC 20024-2628 ...
30365,30366,ricraig,CRAIG,,RICHARD,nan,
578243,578244,jochne,John,C,Neuenschwander,19621228,1508 Ottawa Rd Ann Arbor MI 48105 USA}
182283,182284,changfel,Changfeng,,Liu,19940724,"Room 1401, No. 12, 99 Nong Jinggu East RD. Min..."


### Similarity Measures

In [15]:
# Sample 200 records from df into new df
df_sample = df.sample(100)

In [16]:
df_sample.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
405936,405937,shenx,XiaoPing,,Shen,19630111,2976 S Foxridge Ct Ann Arbor MI 48105-9200 UNI...
165127,165128,salkiek,Sarah,,Alkiek,19991202,6239 Canter Creek Trl Grand Blanc MI 48439-744...
265971,265972,alingamp,Aditya,,Lingampally,19870529,1218 Meadow Sweet Dr Madison WI 53719-4514 UNI...
212137,212138,kgran,Keith,,Gran,19600209,9070 Lebarron Court Saline MI 48176 UNITED STA...
172603,172604,cmohare,Ciara,Marie,O'Hare,19920424,18577 Innsbrook Dr Apt 2 Northville MI 48168-2...


In [17]:
# Remove sampled records from df
df = df.drop(df_sample.index)

With a sample df, we can now build a similarity measure df by comparing each record in the sample df with the other records.

In [18]:
import Levenshtein as lev

In [19]:
"""
Returns normalized levenshtein distance between two strings
"""
def levenshtein_distance(
        s1: str,
        s2: str
) -> int:
    distance = lev.distance(s1, s2)
    try:
        return 1 - distance / float(max(len(s1), len(s2)))
    except ZeroDivisionError:
        return 0

In [20]:
"""
Returns Levenshtein distance between each field of two rows
Assumes following format for rows:
    row = pd.Series([First Name, Middle Name, Last Name, DOB, Address, ID])
"""
def row_similarity(
        row_1: pd.Series, 
        df_2: pd.DataFrame
    ) -> pd.DataFrame:

    # Compute similarity measures for each column using Levenshtein distance
    first_name_similarity = df_2['First Name'].apply(lambda x : levenshtein_distance(row_1['First Name'], x))
    middle_name_similarity = df_2['Middle Name'].apply(lambda x : levenshtein_distance(row_1['Middle Name'], x))
    last_name_similarity = df_2['Last Name'].apply(lambda x : levenshtein_distance(row_1['Last Name'], x))
    dob_similarity = df_2['Date of Birth'].apply(lambda x : levenshtein_distance(row_1['Date of Birth'], x))
    address_similarity = df_2['Address'].apply(lambda x : levenshtein_distance(row_1['Address'], x))
    id_similarity = df_2['ID'].apply(lambda x : levenshtein_distance(row_1['ID'], x))

    # Return DataFrame with similarity measures
    return pd.DataFrame({
        'UNIQ_ID1': row_1['Unique ID'],
        'UNIQ_ID2': df_2['Unique ID'],
        'ID1': row_1['ID'],
        'ID2': df_2['ID'],
        'ID Similarity': id_similarity,
        'First Name 1' : row_1['First Name'],
        'First Name 2' : df_2['First Name'],
        'First Name Similarity': first_name_similarity,
        'Middle Name 1' : row_1['Middle Name'],
        'Middle Name 2' : df_2['Middle Name'],
        'Middle Name Similarity': middle_name_similarity,
        'Last Name 1' : row_1['Last Name'],
        'Last Name 2' : df_2['Last Name'],
        'Last Name Similarity': last_name_similarity,
        'Date of Birth 1' : row_1['Date of Birth'],
        'Date of Birth 2' : df_2['Date of Birth'],
        'Date of Birth Similarity': dob_similarity,
        'Address 1' : row_1['Address'],
        'Address 2' : df_2['Address'],
        'Address Similarity': address_similarity
    })


In [21]:
"""
Builds similarity measure between records in two df
Creates a new df from the two df with the following columns:
    - ID1: ID of record in df1
    - ID2: ID of record in df2
    - First Name Similarity: Normalized levenshtein distance between first names
    - Middle Name Similarity: Normalized levenshtein distance between middle names
    - Last Name Similarity: Normalized levenshtein distance between last names
    - Date of Birth Similarity: Normalized levenshtein distance between dates of birth
    - Address Similarity: Normalized levenshtein distance between addresses
    - ID Similarity: Normalized levenshtein distance between IDs
"""
def build_similarity_df (
        df_1: pd.DataFrame,
        df_2: pd.DataFrame
) -> pd.DataFrame:
    new_df = pd.DataFrame(columns=[ 'UNIQ_ID1',
                                    'UNIQ_ID2',
                                    'ID1', 
                                   'ID2', 
                                   'ID Similarity',
                                   'First Name 1',
                                   'First Name 2',
                                   'First Name Similarity', 
                                   'Middle Name 1',
                                    'Middle Name 2',
                                   'Middle Name Similarity',
                                    'Last Name 1',
                                    'Last Name 2', 
                                   'Last Name Similarity', 
                                    'Date of Birth 1',
                                    'Date of Birth 2',
                                   'Date of Birth Similarity', 
                                    'Address 1',
                                    'Address 2',
                                   'Address Similarity'])

    # Convert ID columns to string
    df_1["ID"] = df_1["ID"].astype(str)
    df_2["ID"] = df_2["ID"].astype(str)

    def apply_row_similarity(row, new_df):
        sim = row_similarity(row, df_2)
        new_df = pd.concat([new_df, sim], ignore_index=True)
        return new_df

    # Use vectorization to compute similarity between each row in df_1 and df_2
    new_df = df_1.apply(apply_row_similarity, args=(new_df,), axis=1).reset_index(drop=True)

    # New df is a series of dfs, so we need to concatenate them
    new_df = pd.concat(new_df.to_list(), ignore_index=True)

    return new_df

In [22]:
similarity_df = build_similarity_df(df_sample,  df)
similarity_df.head()

,UNIQ_ID1,UNIQ_ID2,ID1,ID2,ID Similarity,First Name 1,First Name 2,First Name Similarity,Middle Name 1,Middle Name 2,Middle Name Similarity,Last Name 1,Last Name 2,Last Name Similarity,Date of Birth 1,Date of Birth 2,Date of Birth Similarity,Address 1,Address 2,Address Similarity
0,405937,126042,shenx,dgberton,0.125000,XiaoPing,Dylan,0.125000,,,1.0,Shen,Bertoni,0.142857,19630111,19930625,0.500,2976 S Foxridge Ct Ann Arbor MI 48105-9200 UNI...,837 W Lombard St Baltimore MD 21201 UNITED STA...,0.540984
1,405937,271930,shenx,patwalk,0.000000,XiaoPing,Patrick,0.250000,,J,0.0,Shen,Walker,0.166667,19630111,19950125,0.500,2976 S Foxridge Ct Ann Arbor MI 48105-9200 UNI...,490 M St SW Apt W603 Washington DC 20024-2628 ...,0.437500
2,405937,30366,shenx,ricraig,0.000000,XiaoPing,CRAIG,0.000000,,,1.0,Shen,RICHARD,0.000000,19630111,nan,0.000,2976 S Foxridge Ct Ann Arbor MI 48105-9200 UNI...,,0.016393
3,405937,578244,shenx,jochne,0.166667,XiaoPing,John,0.250000,,C,0.0,Shen,Neuenschwander,0.142857,19630111,19621228,0.375,2976 S Foxridge Ct Ann Arbor MI 48105-9200 UNI...,1508 Ottawa Rd Ann Arbor MI 48105 USA},0.426230
4,405937,182284,shenx,changfel,0.250000,XiaoPing,Changfeng,0.333333,,,1.0,Shen,Liu,0.000000,19630111,19940724,0.375,2976 S Foxridge Ct Ann Arbor MI 48105-9200 UNI...,"Room 1401, No. 12, 99 Nong Jinggu East RD. Min...",0.162791


### Discretization

### WPKID

In [23]:
wpkid_df = similarity_df.copy()

In [24]:
# holds bin ranges for each similarity attribute
bin_ranges = {'First Name Similarity' : [],
        'Middle Name Similarity' : [],
        'Last Name Similarity' : [],
        'Date of Birth Similarity' : [],
        'Address Similarity' : [],
        'ID Similarity' : []}

In [25]:
#for solving systems of equations
import sympy as sp 
import math


t = # of intervals<br>
s = interval size<br>
n = # of instances in training dataset<br>

s * t = n<br>
s - 30 = t<br>


In [26]:
# WPKID function to solve for s and t given n, returns pair {s, t}
def interval_size_and_num(n) -> tuple:
    s, t = sp.symbols('s t')
    equation1 = sp.Eq(s * t, n)
    equation2 = sp.Eq(s - 30, t)  
    solution = sp.solve((equation1, equation2), (s, t))
    s_val = solution[0][1] * -1
    t_val = solution[1][1]

    s_int = math.floor(s_val)
    t_int = math.floor(t_val)
    return s_int, t_int

In [27]:
# function that takes in a column and discretizes it, saving bin ranges
def discretize_column(column, df):
    # separate zeros from nonzeros
    zeros_df = wpkid_df[wpkid_df[column] == 0].copy()
    nonzeros_df = wpkid_df[wpkid_df[column] != 0].copy()

    #get num_instances and num_intervals for nonzeros
    num_instances = interval_size_and_num(len(nonzeros_df))[0]
    num_intervals = interval_size_and_num(len(nonzeros_df))[1]

    # get bin ranges for nonzeros
    nonzeros_df[column], bin = pd.qcut(nonzeros_df[column], 
                                        q=num_intervals, 
                                        labels=False,
                                        retbins=True,
                                        duplicates='drop') 
    
    #add 1 to every value to switch to index 1
    nonzeros_df[column] = nonzeros_df[column] + 1

    #save bin ranges
    bin_ranges[column].extend(bin.tolist())

    #recombine zero and nonzero dataframes
    df.loc[df[column] == 0, column] = zeros_df[column]
    df.loc[df[column] != 0, column] = nonzeros_df[column] 

    return df

In [28]:
wpkid_df = discretize_column('First Name Similarity', wpkid_df)
wpkid_df = discretize_column('Middle Name Similarity', wpkid_df)
wpkid_df = discretize_column('Last Name Similarity', wpkid_df)
wpkid_df = discretize_column('Date of Birth Similarity', wpkid_df)
wpkid_df = discretize_column('Address Similarity', wpkid_df)
wpkid_df = discretize_column('ID Similarity', wpkid_df)

In [29]:
"""insert 0 at the beginning of each bin range"""
for key in bin_ranges:
    bin_ranges[key].insert(0, 0)

In [30]:
wpkid_df.head()

,UNIQ_ID1,UNIQ_ID2,ID1,ID2,ID Similarity,First Name 1,First Name 2,First Name Similarity,Middle Name 1,Middle Name 2,Middle Name Similarity,Last Name 1,Last Name 2,Last Name Similarity,Date of Birth 1,Date of Birth 2,Date of Birth Similarity,Address 1,Address 2,Address Similarity
0,405937,126042,shenx,dgberton,1.0,XiaoPing,Dylan,6.0,,,25.0,Shen,Bertoni,12.0,19630111,19930625,3.0,2976 S Foxridge Ct Ann Arbor MI 48105-9200 UNI...,837 W Lombard St Baltimore MD 21201 UNITED STA...,342.0
1,405937,271930,shenx,patwalk,0.0,XiaoPing,Patrick,15.0,,J,0.0,Shen,Walker,14.0,19630111,19950125,3.0,2976 S Foxridge Ct Ann Arbor MI 48105-9200 UNI...,490 M St SW Apt W603 Washington DC 20024-2628 ...,235.0
2,405937,30366,shenx,ricraig,0.0,XiaoPing,CRAIG,0.0,,,25.0,Shen,RICHARD,0.0,19630111,nan,0.0,2976 S Foxridge Ct Ann Arbor MI 48105-9200 UNI...,,16.0
3,405937,578244,shenx,jochne,2.0,XiaoPing,John,15.0,,C,0.0,Shen,Neuenschwander,12.0,19630111,19621228,2.0,2976 S Foxridge Ct Ann Arbor MI 48105-9200 UNI...,1508 Ottawa Rd Ann Arbor MI 48105 USA},222.0
4,405937,182284,shenx,changfel,4.0,XiaoPing,Changfeng,19.0,,,25.0,Shen,Liu,0.0,19630111,19940724,2.0,2976 S Foxridge Ct Ann Arbor MI 48105-9200 UNI...,"Room 1401, No. 12, 99 Nong Jinggu East RD. Min...",67.0


In [31]:
for key, value in bin_ranges.items():
    print(key, value)

First Name Similarity [0, 0.045454545454545414, 0.0714285714285714, 0.08333333333333337, 0.09090909090909094, 0.09999999999999998, 0.11111111111111116, 0.125, 0.1333333333333333, 0.1428571428571429, 0.15000000000000002, 0.16666666666666663, 0.18181818181818177, 0.19999999999999996, 0.2142857142857143, 0.2222222222222222, 0.25, 0.2727272727272727, 0.2857142857142857, 0.30000000000000004, 0.33333333333333337, 0.36363636363636365, 0.375, 0.4, 0.4285714285714286, 0.4444444444444444, 0.5, 0.5714285714285714, 0.6, 0.625, 0.7142857142857143, 0.8888888888888888, 1.0]
Middle Name Similarity [0, 0.04347826086956519, 0.0625, 0.07692307692307687, 0.08333333333333337, 0.08695652173913049, 0.09090909090909094, 0.09999999999999998, 0.11111111111111116, 0.11764705882352944, 0.125, 0.13043478260869568, 0.1428571428571429, 0.15384615384615385, 0.16666666666666663, 0.19999999999999996, 0.2222222222222222, 0.25, 0.2857142857142857, 0.33333333333333337, 0.375, 0.4, 0.4285714285714286, 0.4444444444444444, 0

In [32]:
# export wpkid_df to csv 
wpkid_df.to_csv('data/sampleunlabeledident.csv', index=False)